In [1]:
import sqlite3

In [ ]:
cursor = conn.cursor()

# Borrar todos los registros de la tabla de catálogo
cursor.execute('DELETE FROM Catalogo')

# Borrar todos los registros de la tabla de proveedores
cursor.execute('DELETE FROM Proveedores')

# Borrar todos los registros de la tabla de partes
cursor.execute('DELETE FROM Partes')

# Guardar cambios y cerrar la conexión
conn.commit()

In [2]:
# Conectar a la base de datos (o crearla si no existe)
conn = sqlite3.connect('C6E2_solucion.db')
cursor = conn.cursor()

# Crear la tabla Proveedores
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Proveedores (
        id INTEGER PRIMARY KEY,
        nombre STRING,
        direccion STRING
    )
''')

# Crear la tabla Partes
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Partes (
        id INTEGER PRIMARY KEY,
        nombre STRING,
        color STRING
    )
''')

# Crear la tabla Catalogo
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Catalogo (
        id_proveedor INTEGER,
        id_parte INTEGER,
        valor REAL,
        FOREIGN KEY (id_proveedor) REFERENCES Proveedores(id),
        FOREIGN KEY (id_parte) REFERENCES Partes(id)
    )
''')

# Guardar los cambios y cerrar la conexión
conn.commit()


In [3]:
with open('catalogo.txt', 'r') as file:
    lines_catalogo = file.readlines()
with open('partes.txt', 'r') as file:
    lines_partes = file.readlines()
with open('proveedores.txt', 'r') as file:
    lines_proveedores = file.readlines()

In [4]:
for line in lines_proveedores:
    data = line.strip().split(',')  # Suponiendo que los datos están separados por comas

    # Insertar datos en la tabla Proveedores
    cursor.execute('INSERT INTO Proveedores (id, nombre, direccion) VALUES (?, ?, ?)', (data[0], data[1], data[2]))
for line in lines_partes:
    data = line.strip().split(',')  # Suponiendo que los datos están separados por comas
    # Insertar datos en la tabla Partes
    cursor.execute('INSERT INTO Partes (id, nombre, color) VALUES (?, ?, ?)', (data[0], data[1], data[2]))
for line in lines_catalogo:
    data = line.strip().split(',')  # Suponiendo que los datos están separados por comas
    # Insertar datos en la tabla Catalogo
    cursor.execute('INSERT INTO Catalogo (id_proveedor, id_parte, valor) VALUES (?, ?, ?)', (data[0], data[1], data[2]))

# Guardar los cambios y cerrar la conexión
conn.commit()

In [5]:
def ejecutar_consulta(cursor, consulta):

    # Ejecutar la consulta
    cursor.execute(consulta)

    # Obtener los resultados (si los hay)
    resultados = cursor.fetchall()

    # Devolver los resultados
    return resultados

In [ ]:
consulta_1 = '''
    SELECT DISTINCT P.nombre
    FROM Proveedores P
    WHERE NOT EXISTS (
        SELECT id
        FROM Partes
        WHERE color = 'Rojo'
        EXCEPT
        SELECT id_parte
        FROM Catalogo C
        WHERE C.id_proveedor = P.id
    );
'''
resultados_1 = ejecutar_consulta(cursor, consulta_1)
print("Resultados 1:")
for resultado in resultados_1:
    print(resultado[0])

Resultados 1:
Acme Widget Suppliers
Big Red Tool and Die
Perfunctory Parts
Alien Aircaft Inc.


In [ ]:
consulta_2 = '''
    SELECT DISTINCT Pa.nombre
    FROM Partes Pa
    JOIN Catalogo C ON Pa.id = C.id_parte
    JOIN Proveedores Pr ON C.id_proveedor = Pr.id
    WHERE Pr.nombre LIKE '%ACME%'
      AND NOT EXISTS (
        SELECT id_proveedor
        FROM Catalogo
        WHERE id_parte = Pa.id
          AND id_proveedor != Pr.id
    );
'''

# Ejecutar la consulta 2
resultados_2 = ejecutar_consulta(cursor, consulta_2)
print("\nResultados 2:")
for resultado in resultados_2:
    print(resultado[0])


Resultados 2:
Acme Widget Washer


In [ ]:
consulta_3 = '''
    SELECT DISTINCT Pr.nombre
    FROM Proveedores Pr
    JOIN Catalogo C ON Pr.id = C.id_proveedor
    WHERE C.valor > (
        SELECT AVG(valor)
        FROM Catalogo
        WHERE id_parte = C.id_parte
    );
'''

# Ejecutar la consulta 3
resultados_3 = ejecutar_consulta(cursor, consulta_3)
print("\nResultados 3:")
for resultado in resultados_3:
    print(resultado[0])


Resultados 3:
Acme Widget Suppliers
Big Red Tool and Die
Perfunctory Parts


In [8]:
consulta_4 = '''
    SELECT Pr.nombre AS nombre_proveedor, Pa.nombre AS nombre_parte, MAX(C.valor) AS precio
    FROM Proveedores Pr
    JOIN Catalogo C ON Pr.id = C.id_proveedor
    JOIN Partes Pa ON C.id_parte = Pa.id
    WHERE Pa.color IN ('Green', 'Red')
    GROUP BY Pr.id, Pa.id;
'''

# Ejecutar la consulta 4
resultados_4 = ejecutar_consulta(cursor, consulta_4)
print("\nResultados 4:")
for resultado in resultados_4:
    print(resultado[0], resultado[1], resultado[2])


Resultados 4:
Acme Widget Suppliers Acme Widget Washer 0.5
Acme Widget Suppliers Fire Hydrant Cap 11.7
Big Red Tool and Die Left Handed Bacon Stretcher Cover 16.5
Big Red Tool and Die Acme Widget Washer 0.55
Big Red Tool and Die Fire Hydrant Cap 7.95
Perfunctory Parts Fire Hydrant Cap 12.5
Perfunctory Parts 7 Segment Display 1.0


In [ ]:
conn.close()